# review로딩 -> df 만들기

In [ ]:
from datetime import datetime
import json
import pandas as pd

# file로딩 -> df 만드는 함수 : 1개의 파일을 전처리하는 함수
def load_data(data_path):
    # 파일로딩
    with open(f"{data_path}", "r", encoding="utf-8") as file:
        json_data = json.load(file)

    # 로딩 데이터 -> df로 만들기
    df = pd.DataFrame(json_data)
    # print(df.head())
    # 데이터 전처리
    df['date'] = df['date'].str.replace(".", "-")
    # date 형태(2025.05.06)가 아닌경우, 오늘날짜로 바꾸기
    # 문자열을 날짜로 변환, 오류가 나는 경우 NaT로 처리
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    # 오늘 날짜
    today = pd.Timestamp(datetime.today().date())
    # NaT인 경우 오늘 날짜로 대체
    df['date'] = df['date'].fillna(today)

    # df 리턴하기
    return df

In [11]:
datetime.today().date()

datetime.date(2025, 6, 5)

In [17]:
# 함수 호출부
data_path = "res/1.gangneung_mare.json"
df = load_data(data_path)
print(len(df))
df.head(10)

420


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20428\2935223973.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


,review,stars,date
0,"바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오...",5,2025-06-05
1,오션뷰라 진짜 좋았어요!! ㅎㅎㅎ\n제 생일이라 남자친구하고 처음 오션뷰 예약해서 ...,5,2025-06-05
2,"황금연휴에 방문했는데 오션뷰가 정말 예쁘더라고요👍\n저는 2층에 머물렀는데, 창밖으...",5,2025-05-06
3,위치좋고 직원들 전부 친절하고 재방문의사100%입니다,5,2025-06-05
4,좋았습니다 수고하세요,5,2025-06-05
5,괜찬아요\n바다바로앞이라좋아요,5,2025-06-05
6,우선 바람이 불어서 예정보다 30분 숙소 입실을 허락해줌\n그런데 오션뷰 2층 20...,2,2025-06-05
7,"강원도를 좋아하는 여행자입니다.\n강릉 스카이베이,속초 카시아,롯데 다이용해봤지만\...",5,2025-05-27
8,강문해변 바로 앞이라 위치도 좋고 버스 정류장도 바로 근처라 이동이 너무 편리해서 ...,5,2025-05-24
9,내부가 깔끔했어요 \n전망도 해안뷰가 이뻤어요\n담에 또 오고싶네요,5,2025-05-24


# mysql DB 연결하기

In [16]:

from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine

# .env 파일을 찾아서 환경변수 로딩
load_dotenv('.env', override=True)

# 메모리에 로딩된 환경변수 값을 가져오기
USER = os.environ.get("USER")
PASSWORD = os.environ.get("PASSWORD")
HOST = os.environ.get("HOST")
DB_NAME = os.environ.get("DB_NAME")
PORT = 3306

# DB_URL = f"mysql+pymysql://{id}:{pw}@{서버ip}:{port}/{db}"
DB_URL = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}"

# df -> mysql 테이블에 저장

In [17]:
def save_to_db(df, table_name):
    try:
        # sqlalchemy로 mysql엔진 생성
        engine = create_engine(DB_URL)    

        # df을 mysql 테이블에 저장
        df.to_sql(
            name = table_name,
            con = engine,
            if_exists = 'append',
            index = False,
            method="multi"
        )
        print(f"SQLAlchemy를 통해 {len(df)}개의 리뷰가 성공적으로 저장되었습니다.")
    except Exception as e:
        print(f"DB 저장시 {e}가 발생함")
    

# 숙소 정보 DB 저장하기

In [32]:
import pandas as pd
df = pd.read_csv('res/accommodation_info.csv', encoding='utf-8')
# df.head()
table_name = 'accommodation_info'
save_to_db(df, table_name)

SQLAlchemy를 통해 4개의 리뷰가 성공적으로 저장되었습니다.


# 리뷰데이터 저장하기

In [35]:
# 전처리
data_path = "res/1.gangneung_mare.json"
df = load_data(data_path)

# df에 accommodation_id 추가하기
# 강릉 마레는 accommodation_id = 1
df['accommodation_id'] = 1
# df.head()

# DB 테이블에 저장하기
table_name = "accommodation_review"
save_to_db(df, table_name)

SQLAlchemy를 통해 420개의 리뷰가 성공적으로 저장되었습니다.


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20428\2935223973.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


# 여러개의 파일을 일괄로 전처하기

- 함수 정의

In [38]:
from datetime import datetime
import json
import pandas as pd

# file로딩 -> df 만드는 함수 : 1개의 파일을 전처리하는 함수
def load_data_multi(acco_id, data_path):
    # 파일로딩
    with open(f"{data_path}", "r", encoding="utf-8") as file:
        json_data = json.load(file)

    # 로딩 데이터 -> df로 만들기
    df = pd.DataFrame(json_data)
    # print(df.head())
    # 데이터 전처리
    df['date'] = df['date'].str.replace(".", "-")
    # date 형태(2025.05.06)가 아닌경우, 오늘날짜로 바꾸기
    # 문자열을 날짜로 변환, 오류가 나는 경우 NaT로 처리
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    # 오늘 날짜
    today = pd.Timestamp(datetime.today().date())
    # NaT인 경우 오늘 날짜로 대체
    df['date'] = df['date'].fillna(today)

    # df에 accommodation_id 추가하기
    # 강릉 마레는 accommodation_id = 1
    df['accommodation_id'] = acco_id
    # df.head()

    # df 리턴하기
    return df

- 일괄 실행

In [39]:
# 전처리하고 저장해야하는 데이터 파일이 10
MAPPING = {
    # 1: './res/1.gangneung_mare.json',
    2: './res/2.insadong_ninetree.json',
    3: './res/3.kyongpo_the_hongc.json',
    # 4: './res/4.hongcheon_vivaldi.json'
}

for acco_id, data_path in MAPPING.items():
    # 전처리
    # data_path = "res/1.gangneung_mare.json"
    df = load_data_multi(acco_id, data_path)

    # DB 테이블에 저장하기
    table_name = "accommodation_review"
    save_to_db(df, table_name)

SQLAlchemy를 통해 420개의 리뷰가 성공적으로 저장되었습니다.
SQLAlchemy를 통해 420개의 리뷰가 성공적으로 저장되었습니다.


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20428\1316262341.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


# review summary 한결과 DB 저장하기

- 리뷰 데이터 로딩
- 프롬프트를 활용해서, LLM을 통한 리뷰요약
- 요약한 결과를 dB 저장하기

In [7]:
from summary_modules import load_prompt, preprocess_reviews, summarize

In [8]:
promt_file = "res/prompt_1shotv1.pickle"
prompt = load_prompt(promt_file)
prompt

'당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.\n\n요약 결과는 다음 조건들을 충족해야 합니다:\n1. 모든 문장은 항상 존댓말로 끝나야 합니다.\n2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.\n  2-1. 좋은 예시\n    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.\n    b) 재방문 예정이라는 평들이 존재합니다.\n  2-2. 나쁜 예시\n    a) 좋은 숙소였고 방음도 괜찮았습니다.\n    b) 재방문 예정입니다.\n3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.\n\n다음은 리뷰들과 요약 예시입니다.\n예시 리뷰들:\n[REVIEW_START]바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오는것도 좋았어요~ 친구들이랑 왔다가 다들 만족하고 갑니다ㅎ[REVIEW_END]\n[REVIEW_START]오션뷰라 진짜 좋았어요!! ㅎㅎㅎ\n제 생일이라 남자친구하고 처음 오션뷰 예약해서 온거였는데 진짜 색달랐어요. 옆에 바다보면서 닭강정먹고.. 또 새벽에 바다 구경하고.. 진짜 잘 보내다 가요🥰[REVIEW_END]\n[REVIEW_START]황금연휴에 방문했는데 오션뷰가 정말 예쁘더라고요👍\n저는 2층에 머물렀는데, 창밖으로 보이는 바다가 너무 아름다웠어요\n직원분들도 모두 친절하셔서 기분 좋게 머물다 갑니다👏\n다음에도 꼭 다시 방문하고 싶어요![REVIEW_END]\n[REVIEW_START]강원도를 좋아하는 여행자입니다.\n강릉 스카이베이,속초 카시아,롯데 다이용해봤지만\n진짜 이가격에 이정도 퀄리티숙소 너무너무 만족하며 이용했습니다\n침구며 객실관리가 청결하게 잘 관리되어있고\n침대도 푹신하니 좋았어요\n단점이라고 하면 이층이라그런가 밤늦게까지 폭죽소리가 너무 시끄러웠고\n비게가 저한테는 높아서 좀 불편했다는 정도..?\n이가격에 오션뷰에 호텔급 침구에 너무 만족했습니다\n재방문의사 100프로입니다[REVIEW_END]\n[

In [ ]:
# 전처리
data_path = "res/1.gangneung_mare.json"
reviews_good, reviews_bad = preprocess_reviews(data_path)

In [11]:
reviews_good

'[REVIEW_START]바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오는것도 좋았어요~ 친구들이랑 왔다가 다들 만족하고 갑니다ㅎ[REVIEW_END]\n[REVIEW_START]오션뷰라 진짜 좋았어요!! ㅎㅎㅎ\n제 생일이라 남자친구하고 처음 오션뷰 예약해서 온거였는데 진짜 색달랐어요. 옆에 바다보면서 닭강정먹고.. 또 새벽에 바다 구경하고.. 진짜 잘 보내다 가요🥰[REVIEW_END]\n[REVIEW_START]황금연휴에 방문했는데 오션뷰가 정말 예쁘더라고요👍\n저는 2층에 머물렀는데, 창밖으로 보이는 바다가 너무 아름다웠어요\n직원분들도 모두 친절하셔서 기분 좋게 머물다 갑니다👏\n다음에도 꼭 다시 방문하고 싶어요![REVIEW_END]\n[REVIEW_START]강원도를 좋아하는 여행자입니다.\n강릉 스카이베이,속초 카시아,롯데 다이용해봤지만\n진짜 이가격에 이정도 퀄리티숙소 너무너무 만족하며 이용했습니다\n침구며 객실관리가 청결하게 잘 관리되어있고\n침대도 푹신하니 좋았어요\n단점이라고 하면 이층이라그런가 밤늦게까지 폭죽소리가 너무 시끄러웠고\n비게가 저한테는 높아서 좀 불편했다는 정도..?\n이가격에 오션뷰에 호텔급 침구에 너무 만족했습니다\n재방문의사 100프로입니다[REVIEW_END]\n[REVIEW_START]강문해변 바로 앞이라 위치도 좋고 버스 정류장도 바로 근처라 이동이 너무 편리해서 좋았습니다. 객실과 화장실도 매우 청결 했어요! 너무 쾌적하게 잘 지내다 갑니다[REVIEW_END]\n[REVIEW_START]내부가 깔끔했어요 \n전망도 해안뷰가 이뻤어요\n담에 또 오고싶네요[REVIEW_END]\n[REVIEW_START]바다소리들으며 편하게잤습니다 ㅎ\n금연객실에ㆍ 회ㆍ대게 종류 반입이안되는데 그만큼관리하시고신경써서그런지 객실이너무청결해서 정말좋았습니다 \n친절한응대도 👍 👍 👍 👍 👍 👍 👍 👍[REVIEW_END]\n[REVIEW_START]오션뷰가 너무 예쁘고 방도 깨끗했습니다. 강릉 갈

In [ ]:
summary_good= summarize(reviews_good, prompt)
# summary_bad= summarize(reviews_bad, prompt)

In [10]:
summary_good

'바닷가에 위치한 숙소로, 깨끗하고 쾌적한 객실입니다. 샤워기 수압이 좋고 따뜻한 물이 잘 나오는 것도 좋은 포인트입니다. 친구들과 함께 머물기에 만족스러운 곳입니다.\n오션뷰가 아름다운 숙소로, 생일을 즐기기에 좋은 곳입니다. 바다를 바라보며 특별한 시간을 보낼 수 있어요. 직원분들의 친절한 응대와 아름다운 바다 전망이 인상적입니다.재방문 의사가 100%입니다.\n황금연휴에 머물렀던 숙소는 오션뷰가 아름다웠고, 2층에서 바라보는 바다 풍경이 아름다웠습니다. 친절한 직원분들과 청결한 객실로 기분 좋게 머물었습니다. 다음에도 꼭 다시 방문하고 싶은 곳입니다.'

# 모든 리뷰 요약 데이터 일괄 처리
- 전처리, 요약하기, df 만들기, mysql에 저장

In [ ]:
# 리뷰 데이터 전처리
MAPPING = {
    1: './res/1.gangneung_mare.json',
    2: './res/2.insadong_ninetree.json',
    3: './res/3.kyongpo_the_hongc.json',
    # 4: './res/4.hongcheon_vivaldi.json'
}


# 리뷰 요약결과 DB에 저장한 사람은 화면 캡쳐해서 디스코드 랜덤에 올리기

In [18]:
import os
import pandas as pd
from summary_modules import preprocess_reviews, summarize, load_prompt

# 베스트 프롬프트 로딩
promt_file = "res/prompt_1shotv1.pickle"
prompt = load_prompt(promt_file)

# 리뷰 데이터 전처리
MAPPING = {
    1: './res/1.gangneung_mare.json',
    2: './res/2.insadong_ninetree.json',
    3: './res/3.kyongpo_the_hongc.json',
    # 4: './res/4.hongcheon_vivaldi.json'
}
table_name='review_summary'

# 리뷰 전처리
for id, path in MAPPING.items():
    reviews_good, reviews_bad = preprocess_reviews(path)

    if reviews_good:
        summary_good = summarize(reviews_good, prompt)
        # print(summary_good)
    else:
        summary_good = "분석할 높은 평점 리뷰가 없습니다."
    # 나쁜 리뷰 요약
    if reviews_bad:
        summary_bad = summarize(reviews_bad, prompt)
        # print(summary_bad)
    else:
        summary_bad = "분석할 낮은 평점 리뷰가 없습니다."

    # 처리결과 데이터 프레임으로 만들어서 저장하기
    df = pd.DataFrame({
        'accommodation_id': [id],
        'summary_good': [summary_good],
        'summary_bad': [summary_bad],
        'date': [pd.Timestamp.now().strftime('%Y-%m-%d')]
    })
    print(df)

    # # MySQL에 저장    
    save_to_db(df, table_name)
    print(f"{path} 숙소의 리뷰 요약이 완료되었습니다.")

   accommodation_id                                       summary_good  \
0                 1  바닷가에 위치한 숙소로, 깨끗하고 쾌적한 객실입니다. 오션뷰를 즐길 수 있는 곳으로...   

                                         summary_bad        date  
0  숙소는 깨끗하고 주차장이 넓어 편리합니다. 화장실 배수가 잘 안되고 샤워하면 역류되...  2025-06-05  
SQLAlchemy를 통해 1개의 리뷰가 성공적으로 저장되었습니다.
./res/1.gangneung_mare.json 숙소의 리뷰 요약이 완료되었습니다.
   accommodation_id                                       summary_good  \
0                 2  깔끔하고 생각보다 넓어서 더 쾌적한 숙소입니다. 근처에 다른 볼거리가 많아서 스테이...   

                                         summary_bad        date  
0  컨디션 양호하고 직원분들이 친절하며, 방음이 조금 아쉬운 패밀리객실입니다. 아이들이...  2025-06-05  
SQLAlchemy를 통해 1개의 리뷰가 성공적으로 저장되었습니다.
./res/2.insadong_ninetree.json 숙소의 리뷰 요약이 완료되었습니다.
   accommodation_id                                       summary_good  \
0                 3  강릉 여행길에 들렀는데 아주 예약을 잘한 숙소였습니다. 주인의 품격이 느껴지고 조용...   

                                         summary_bad        date  
0  공사로 인해 불편함을 느꼈던 숙소입니다. 예약할 때부터 공사 안내가 필요